In [1]:

import pandas as pd
import numpy as np


initial_data = pd.read_csv("Snappfood.csv", sep="\t", encoding="utf-8-sig")
filtered_data = initial_data.dropna(subset=["comment", "label", "label_id"])
new_data = pd.DataFrame()
new_data["comment"] = filtered_data["comment"].dropna().values
new_data["label"] = filtered_data["label"].str.lower().dropna().values
new_data["label_id"] = filtered_data["label_id"].dropna().values.astype(np.int8)

new_data.to_csv("new_data.csv",encoding="utf-8-sig" ,index=False)

